# Beam-Column

TODO: Uniaxial vs Biaxial

## Setup

### Imports & Env. Settings

In [1]:
# Imports
import handcalcs # https://github.com/connorferster/handcalcs/wiki
from handcalcs import render # https://github.com/connorferster/handcalcs?tab=readme-ov-file#basic-usage-1-as-a-jupyter-cell-magic-render
import forallpeople as si # https://connorferster.github.io/forallpeople/basic_usage.html
from modules.section import UniversalColumn
from modules.strength import design_strength

In [2]:
# Environment config.
# --> SI units structural env ( https://connorferster.github.io/forallpeople/environments.html#Environments )
si.environment("structural", top_level=True) # load SI units for structural eng. into the environment, top_level=True defines them in the top-level namespace (as vars)
# -> Handcalcs environment settings for TeX rendering
handcalcs.set_option("latex_block_start", "$")
handcalcs.set_option("latex_block_end", "$")
handcalcs.set_option("math_environment_start", "aligned")
handcalcs.set_option("math_environment_end", "aligned")

### Question Data

In [3]:
%%render params 2 # Loads/Moments, lengths, qn. params
# Actions
F_c = 2000 * kN
M_x = 100 * kN * m
M_y = 20 * kN * m

# Dimensions
L = 10 * m

# Strengths
p_y = 265 * MPa # 📝

<IPython.core.display.Latex object>

### Trial Section

In [4]:
trial_section = UniversalColumn.create(designation="305x305x118")
# ... UB not implemented, should be in rework of sections module

In [5]:
%%render params 1 # BS Notation
D = trial_section.D * mm
B = trial_section.B * mm
d = trial_section.d * mm
t = trial_section.t * mm
T = trial_section.T * mm

S_xx = trial_section.S_xx * 1000*mm**3
S_yy = trial_section.S_yy * 1000*mm**3
Z_xx = trial_section.Z_xx * 1000*mm**3
Z_yy = trial_section.Z_yy * 1000*mm**3
r_xx = trial_section.r_xx * 10*mm
r_yy = trial_section.r_yy * 10*mm
u = trial_section.u
x = trial_section.x
A = trial_section.A * 100*mm**2

<IPython.core.display.Latex object>

<!-- ### Classification -->

### Effective Length $L_{\text{E}}$ / $L_{\text{Cr}}$

Table 22 BS 5950
_ENV 1993-1-1:2004_

In [6]:
%%render short 2 # Uncomment correct value, defaults to 1.0 L
L_E = 0.7 * L # 📝
# L_E = 0.85 * L
# L_E = 1.0 * L
# L_E = 1.2 * L
# L_E = 1.5 * L
# L_E = 2.0 * L

# L_Cr = L_E

<IPython.core.display.Latex object>

## BS 5950

### Classification

In [7]:
# Strength class (Defaults to S275) Table 9 BS
# thickest = max(trial_section.t, trial_section.T)
# p_yd = design_strength(thickness=thickest, steel_grade=275)
# print(f"Thickness: {thickest} mm")
# print(f"Design strength: {p_yd} MPa")

In [8]:
# Section Class Table 11 BS

### Local Capacity

In [9]:
%%render short 3 # Capacities Pc, Mcx and Mcy
P_c = A * p_y
M_cx = S_xx * p_y
M_cy = S_yy * p_y

<IPython.core.display.Latex object>

In [10]:
%%render short 2 # Local Capacity Check, Cl. 4.8.3.2
# util is utilisation ratio
util = F_c/P_c + M_x/M_cx + M_y/M_cy
# util = (F_c/P_c) + (M_x/M_cx) + (M_y/M_cy)

if util < 1: LocalCapacity = "OK✅"
elif util >= 1: LocalCapacity = "INADEQUATE🚫"

# ... raw
# if F_c/P_c + M_x/M_cx + M_y/M_cy < 1: LocalCapacity = "OK✅"
# elif F_c/P_c + M_x/M_cx + M_y/M_cy >= 1: LocalCapacity = "INADEQUATE🚫"

<IPython.core.display.Latex object>

### In-Plane (Flexural) Buckling

#### $\lambda$, Strut Curve and $p_{\text{cx}}$ & $p_{\text{cy}}$

In [11]:
%%render params 2
L_E
r_yy
lamb = L_E / r_yy

# ... read off pc from table 24
p_c = 139 * MPa # 📝

<IPython.core.display.Latex object>

#### $P_{\text{c}}$

In [12]:
%%render short 3
P_c = A * p_c

<IPython.core.display.Latex object>

#### $m$

In [13]:
%%render params 2 # r.... defaults to 1
m = 1.0 # 📝
m_x = m # 📝
m_y = m # 📝

<IPython.core.display.Latex object>

#### $M_{\text{cx}}$ and $M_{\text{cy}}$

In [14]:
%%render short 2 # unnec. for check; check calcs. products
# M_cx_ip viz. in-plane, != M_cx (local) i.e plastic vs elastic moduli
M_cx_ip = p_y * Z_xx
M_cy_ip = p_y * Z_yy

<IPython.core.display.Latex object>

#### Check

In [15]:
%%render short 2 # In-Plane Buckling Check, Cl. 4.
# util_ipb is utilisation ratio for in-plane buckling
util_ipb = F_c/P_c + (m_x*M_x)/(p_y*Z_xx) + (m_y*M_y)/(p_y*Z_yy)
# util_ipb = (F_c/P_c) + ((m_x*M_x)/(p_y*Z_xx)) + ((m_y*M_y)/(p_y*Z_yy))

if util_ipb < 1: InPlaneBucklingCapacity = "OK✅"
elif util_ipb >= 1: InPlaneBucklingCapacity = "INADEQUATE🚫"

# ... raw
# if F_c/P_c + (m_x*M_x)/(p_y*Z_xx) + (m_y*M_y)/(p_y*Z_yy) < 1: InPlaneBucklingCapacity = "OK✅"
# elif F_c/P_c + M_x/(p_y*Z_xx) + M_y/(p_y*Z_yy) >= 1: InPlaneBucklingCapacity = "INADEQUATE🚫"

<IPython.core.display.Latex object>

### Lateral-Torsional Buckling

#### $\nu$ and $\lambda_{\text{LT}}$

In [16]:
%%render params 2
beta = 1.0 
u

<IPython.core.display.Latex object>

In [17]:
%%render short 3
nu = 1/( (1 + 0.05 * (lamb/x)**2)**(0.25) )
lamb_LT = u * nu * lamb * beta**0.5

<IPython.core.display.Latex object>

#### $p_{\text{b}}$

In [18]:
%%render params 2
p_b = 196 * MPa # 📝

<IPython.core.display.Latex object>

#### $m_{\text{LT}}$ and $M_{\text{LT}}$

In [19]:
%%render params 2 # m_LT read from table 18; M_LT is either Mx or max(Mx, My)
m_LT = 1.0 # 📝
M_LT = M_x
# M_LT = max(M_x, M_y)

<IPython.core.display.Latex object>

#### $M_{\text{b}}$

In [20]:
%%render short 2
M_b = S_xx * p_b

<IPython.core.display.Latex object>

### Check...

In [21]:
%%render short 2 # LTB Capacity Check, Cl. 4.8.3.2
# util_ltb is utilisation ratio
util_ltb = F_c/P_c + (m_LT*M_LT)/M_b + M_y/(p_y*Z_yy)
# util_ltb = (F_c/P_c) + ((m_LT*M_LT)/M_b) + (M_y/(p_y*Z_yy))

if util_ltb < 1: LTBucklingCapacity = "OK✅"
elif util_ltb >= 1: LTBucklingCapacity = "INADEQUATE🚫"

# ... raw
# if F_c/P_c + (m_LT*M_LT)/M_b + M_y/(p_y*Z_yy)< 1: LTBucklingCapacity = "OK✅"
# elif F_c/P_c + (m_LT*M_LT)/M_b + M_y/(p_y*Z_yy) >= 1: LTBucklingCapacity = "INADEQUATE🚫"

<IPython.core.display.Latex object>

## Eurocode 3

### Params

In [22]:
%%render params 1 # EC3 Notation
h = D
b = B
t_w = t
t_f = T

W_ply = S_xx
W_plz = S_yy
W_ely = Z_xx
W_elz = Z_yy
i_yy = r_xx
i_zz = r_yy
U = u
X = x
A = trial_section.A * 100*mm**2
I_yy = trial_section.I_xx * 10000*mm**4
I_zz = trial_section.I_yy * 10000*mm**4

<IPython.core.display.Latex object>

In [23]:
%%render params 2 # ...TODO: check, incl. fy
N_Ed = F_c
M_yEd = M_x
M_zEd = M_y
f_y = p_y

# --
E = 210 * GPa
gamma_m_0 = 1.0
gamma_m_1 = 1.0
pi = 3.14159265359

<IPython.core.display.Latex object>

### $M_{\text{N,Rd}}$

#### $N_{\text{Ed}}$ vs $0.25N_{\text{pl,Rd}}$ & $0.5h_{\text{w}}t_{\text{w}}f_{\text{y}}$

In [24]:
%%render  2 # reduce NEd is less than the smaller of 0.25NplRd or half the web's plastic tension resistance
N_plRd = A * f_y / gamma_m_0
h_w = h - 2*t_f
N_webaxial = h_w * t_w * f_y / gamma_m_0

# ... safer
# N_prime_prime_1 = 0.25 * N_plRd
# N_prime_prime_2 = 0.5 * N_webaxial
# # N_prime = min(N_prime_prime_1, N_prime_prime_2)

# ... raw
N_prime = min(0.25*N_plRd, 0.5*h_w*t_w*f_y)

<IPython.core.display.Latex object>

#### $n$, $a$, $M_{\text{pl,y,Rd}}$ and $M_{\text{pl,z,Rd}}$

In [25]:
%%render short 3
n = N_Ed / N_plRd
a = (A - 2 * b * t_f)/A # 📝 or ifelse for a<=0.5
M_plyRd = W_ply * f_y / gamma_m_0
M_plzRd = W_plz * f_y / gamma_m_0

<IPython.core.display.Latex object>

#### $M_{\text{N,y,Rd}}$ and $M_{\text{N,z,Rd}}$

In [26]:
%%render short 2 # 
if N_Ed < N_prime: M_NyRd = M_plyRd
elif N_Ed >= N_prime: M_NyRd = M_plyRd * (1 - n)/(1 - 0.5*a)

<IPython.core.display.Latex object>

In [27]:
# %%render short 2 #! INCOMPLETE check logic for N_Ed > Nweb
# # if N_Ed <= (h_w * t_w * f_y): M_NzRd = M_plzRd #! KeyError
# if N_Ed < N_webaxial: M_NzRd = M_plzRd

In [28]:
%%render short 2  #! INCOMPLETE check logic for N_Ed > Nweb and M_NzRd < MplRd as max. val.
# # if N_Ed <= (h_w * t_w * f_y): M_NzRd = M_plzRd
# M_NzRd = 1 # 🩹 hotfix for KeyError, quite weird...
if N_Ed < N_webaxial: M_NzRd = M_plzRd 
elif N_Ed >= N_webaxial: check_n_a = True

if n <= a: M_NzRd = M_plzRd
elif n > a: M_NzRd = M_plzRd * (1 - ( (n-a)/(1-a) )**2)

<IPython.core.display.Latex object>

### $M_{\text{b,Rd}}$

#### $N_{\text{Cr}}$ and $\lambda_{\text{y}}$

In [29]:
%%render short 2
L_Cr = L_E * m # 📝 # FIXME
L_Cry = L_Cr # 📝
L_Crz = L_Cr # 📝
N_Cry = (pi**2 * E * I_yy)/(L_Cry**2)
lamb_y = ( (A*f_y)/N_Cry )**0.5

<IPython.core.display.Latex object>

### $\Phi_{\text{y}}$ and $\chi_{\text{y}}$

In [ ]:
%%render short 2 # having selected alpha from ...
alpha = 0.49
Phi = 0.5 * (1 + alpha*(lamb_z -0.2) + lamb_z**2)
chi = 1 / (Phi + (Phi**2 - lamb_z**2)**0.5)